In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from collections import defaultdict


Importation des données

In [9]:
# Transformations pour normaliser les images et les redimensionner
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionner les images
    transforms.ToTensor(),          # Convertir en tenseur PyTorch
    transforms.Normalize(mean=[0.5, 0.5, 0.5],  # Normalisation
                         std=[0.5, 0.5, 0.5])
])

# Charger les données avec ImageFolder
dataset = datasets.ImageFolder(root="data_img", transform=transform)

# Créer un DataLoader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Obtenir la correspondance des classes
print("Classes:", dataset.classes)
print("Index des classes:", dataset.class_to_idx)

Classes: ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'ragno', 'sheep', 'squirrel']
Index des classes: {'butterfly': 0, 'cat': 1, 'chicken': 2, 'cow': 3, 'dog': 4, 'elephant': 5, 'horse': 6, 'ragno': 7, 'sheep': 8, 'squirrel': 9}


Sélectionner un échantillon

In [10]:
# Obtenir la correspondance des classes
print("Classes:", dataset.classes)
print("Index des classes:", dataset.class_to_idx)

# Initialiser un dictionnaire pour stocker les images et labels
class_images = defaultdict(list)
class_labels = defaultdict(list)

# Limiter à 100 images par classe
max_images_per_class = 10

# Parcourir les lots d'images
for images, labels in data_loader:
    for image, label in zip(images, labels):
        class_images[label.item()].append(image)
        class_labels[label.item()].append(label.item())

        # Arrêter dès qu'on a 100 images pour chaque classe
        if all(len(class_images[class_idx]) >= max_images_per_class for class_idx in class_images):
            break
    else:
        continue  # Si break n'a pas été appelé, continuer à parcourir le batch
    break  # Sortir de la boucle principale quand on a atteint 100 images pour chaque classe

# Limiter à 10 images pour chaque classe
for class_idx in class_images:
    class_images[class_idx] = class_images[class_idx][:max_images_per_class]
    class_labels[class_idx] = class_labels[class_idx][:max_images_per_class]

# Afficher la forme des images et labels pour vérifier
print(f"Nombre d'images par classe :")
for class_idx, images in class_images.items():
    print(f"Classe {dataset.classes[class_idx]} : {len(images)} images")


Classes: ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'ragno', 'sheep', 'squirrel']
Index des classes: {'butterfly': 0, 'cat': 1, 'chicken': 2, 'cow': 3, 'dog': 4, 'elephant': 5, 'horse': 6, 'ragno': 7, 'sheep': 8, 'squirrel': 9}
Nombre d'images par classe :
Classe sheep : 10 images
Classe cat : 10 images
Classe squirrel : 10 images
Classe dog : 10 images
Classe butterfly : 10 images
Classe elephant : 10 images
Classe ragno : 10 images
Classe cow : 10 images
Classe horse : 10 images
Classe chicken : 10 images


<h3> Créer un nouveau dataset <h3/>

In [11]:
import torch
from torch.utils.data import Dataset

class SubsetDataset(Dataset):
    def __init__(self, class_images, class_labels):
        self.class_images = class_images
        self.class_labels = class_labels
        # Créer une liste de toutes les images et labels à partir des dictionnaires
        self.images = []
        self.labels = []
        for label, images in class_images.items():
            self.images.extend(images)
            self.labels.extend([label] * len(images))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

# Créer un nouveau dataset avec les images collectées
subset_dataset = SubsetDataset(class_images, class_labels)

# Créer un DataLoader pour ce subset
subset_loader = torch.utils.data.DataLoader(subset_dataset, batch_size=32, shuffle=True)

# Afficher un exemple de lot d'images et labels
for images, labels in subset_loader:
    print(images.shape, labels)
    break


torch.Size([32, 3, 128, 128]) tensor([0, 7, 4, 1, 6, 3, 8, 8, 2, 4, 3, 2, 9, 3, 0, 4, 2, 5, 8, 0, 0, 2, 6, 5,
        5, 7, 7, 4, 8, 9, 2, 0])


In [16]:
from torch.utils.data import Dataset, random_split

train_size = int(0.8 * len(subset_dataset))  # 80% pour l'entraînement
valid_size = len(subset_dataset) - train_size  # 20% pour la validation

# Diviser le dataset en deux partitions : train et valid
train_dataset, valid_dataset = random_split(subset_dataset, [train_size, valid_size])

# Créer un DataLoader pour ces data
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=True)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
import matplotlib.pyplot as plt
import numpy as np

In [12]:
# === 2. Définition du modèle CNN ===
class CNN_Animals(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # --- Couches convolutionnelles ---
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)

        # --- Couches fully connected ---
        # Après 3 convolutions + 3 poolings : 128x128 → 16x16
        self.fc1 = nn.Linear(64 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)  # aplatir
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# === 3. Initialisation du modèle ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_Animals(num_classes=10).to(device)
print(model)

# === 4. Définition de la fonction de perte et de l’optimiseur ===
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# === 5. Fonction d'entraînement ===
def train_loop(loader, model, loss_fn, optimizer, log=True):
    size = len(loader.dataset)
    model.train()
    total_loss = 0

    for batch, (X, y) in enumerate(loader):
        X, y = X.to(device), y.to(device)

        # forward
        pred = model(X)
        loss = loss_fn(pred, y)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if log and batch % 10 == 0:
            current = (batch + 1) * len(X)
            print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")

    avg_loss = total_loss / len(loader)
    print(f"🔹 Moyenne de la loss sur l'époque : {avg_loss:.4f}")
    return avg_loss



CNN_Animals(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=16384, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [18]:
# === 6. Lancer l'entraînement (ex : 3 époques) ===
for epoch in range(10):
    print(f"\n===== Époque {epoch+1} =====")
    train_loop(train_loader, model, loss_fn, optimizer)


===== Époque 1 =====
loss: 1.755813  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 1.5694

===== Époque 2 =====
loss: 1.215848  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 1.2078

===== Époque 3 =====
loss: 0.887923  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 1.0405

===== Époque 4 =====
loss: 0.734020  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.9823

===== Époque 5 =====
loss: 0.600018  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.6574

===== Époque 6 =====
loss: 0.662158  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.6488

===== Époque 7 =====
loss: 0.297627  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.4045

===== Époque 8 =====
loss: 0.408294  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.3614

===== Époque 9 =====
loss: 0.118275  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.1510

===== Époque 10 =====
loss: 0.102871  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.1206


<h3> Evaluation du modèle <h3/>

In [ ]:
if 'model' in locals():
    del model

In [14]:
def test_loop(loader, model, loss_fn, log=True):
    model.eval() #evaluating the model
    size = len(loader.dataset)
    num_batches = len(loader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    with torch.no_grad():
        for X, y in loader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    if log:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, loss: {test_loss:>8f} \n")
    return test_loss, 100*correct

In [ ]:
epochs = 10
loss_test = []
loss_train = []
acc_test = []

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"\n===== Époque {t+1} =====")
    loss_train.append(train_loop(train_loader, model, loss_fn, optimizer))
    l, a = test_loop(valid_loader, model, loss_fn)
    loss_test.append(l)
    acc_test.append(a)
print("Done!")

loss: 0.049954  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.0680
Test Error: 
 Accuracy: 25.0%, loss: 4.522668 

loss: 0.255205  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.1357
Test Error: 
 Accuracy: 15.0%, loss: 5.070909 

loss: 0.133264  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.0855
Test Error: 
 Accuracy: 10.0%, loss: 5.968122 

loss: 0.292206  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.1518
Test Error: 
 Accuracy: 35.0%, loss: 4.524742 

loss: 0.053805  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.1058
Test Error: 
 Accuracy: 30.0%, loss: 4.098357 

loss: 0.016111  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.0765
Test Error: 
 Accuracy: 20.0%, loss: 4.510278 

loss: 0.039718  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.2048
Test Error: 
 Accuracy: 15.0%, loss: 4.788084 

loss: 0.038378  [   32/   80]
🔹 Moyenne de la loss sur l'époque : 0.0393
Test Error: 
 Accuracy: 15.0%, loss: 4.971359 

loss: 0.030310  [   32/   80]
🔹 